# NASA Earthdata Worldview

https://worldview.earthdata.nasa.gov/?p=geographic&l=SMAP_L4_Emult_Average&t=2018-08-01-T00%3A00%3A00Z&z=3&v=-180,-90,180,90

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
import pandas as pd
from imageio import mimwrite
import requests

In [ ]:
start_date = '2015-01-01'
end_date = '2019-01-01'

In [ ]:
timeline = pd.date_range(start_date, end_date, freq='3MS')

for t in timeline:
    print(t.strftime('%Y-%m-%d'))

In [ ]:
layer = 'SMAP_L4_Emult_Average'

In [ ]:
def collect_img_binary(date, layer, extent, width, height):
    
    # Create URL to download image
    nasa_url = "https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot"
    nasa_url += "&TIME=%s" % date.strftime('%Y-%m-%d')
    nasa_url += "&BBOX=%s" % ','.join([str(e) for e in extent])
    nasa_url += "&CRS=EPSG:4326"
    nasa_url += "&LAYERS=%s" % layer
    nasa_url += "&FORMAT=image/png"
    nasa_url += "&WIDTH={0:d}&HEIGHT={1:d}".format(width, height)
    
    # Request content and save binary PNG file in variable
    r = requests.get(nasa_url, allow_redirects=True)
    img_binary = r.content
    
    return img_binary

In [ ]:
def recreate_img(img_binary, height):
    
    # Store binary content in PNG file 'temp.png'
    with open('temp.png', 'wb') as f:
        f.write(img_binary)

    # Open PNG file and extract data value for each RGBA pixel
    im_frame = Image.open('temp.png')
    data = np.array(im_frame.getdata()).reshape(height, -1, 4)
    
    return data

In [ ]:
def collect_dataset(start_date, end_date, layer, extent=(27, -10, 63, 43),
                    width=400, frequency='1MS'):
    
    # Compute height based on width and the bounding box of the image
    relation = float(extent[2] - extent[0]) / (extent[3] - extent[1])
    height = int(width * relation)
    
    timeline = pd.date_range(start_date, end_date, freq=frequency)
    
    dataset = []
    
    for t in timeline:
        
        img_binary = collect_img_binary(t.date(), layer, extent, width, height)
    
        data = recreate_img(img_binary, height)

        data = data[..., :3].astype('uint8')

        dataset.append(data)

        print(t.strftime('%Y-%m-%d'))
    
    return np.array(dataset)

In [ ]:
def img_has_content(data, thresh=0.15):
    rows_with_content = np.where(np.sum(data, axis=(-2, -1)))
    return np.max(rows_with_content) > data.shape[1]*thresh

In [ ]:
dataset = collect_dataset(start_date, end_date, layer)

In [ ]:
img_has_content(dataset[0], 0.15)

In [ ]:
plt.figure(figsize=(12, 10))
plt.imshow(dataset[0])

In [ ]:
layers=['SMAP_L4_Emult_Average',
        'MODIS_Terra_EVI_8Day',
        'SMAP_L4_Soil_Temperature_Layer_1',
        'SMAP_L2_Passive_Enhanced_Day_Soil_Moisture_Option1',
        'SMAP_L4_Uncertainty_Mean_Net_Ecosystem_Exchange',
        'SMAP_L4_Mean_Net_Ecosystem_Exchange',
        'SMAP_L4_Mean_Heterotrophic_Respiration',
        'MODIS_Combined_Thermal_Anomalies_All',
        'MODIS_Terra_Chlorophyll_A',
        'MODIS_Aqua_Chlorophyll_A',
        'SMAP_L1_Passive_Enhanced_Brightness_Temp_Fore_H',
        'AMSUA_NOAA15_Brightness_Temp_Channel_1',
        'VIIRS_SNPP_CorrectedReflectance_TrueColor'
        ]

In [ ]:
start_date = '2015-01-01'
end_date = '2019-01-01'

for l in layers:
    print('\n', l)
    dataset = collect_dataset(start_date, end_date, l, width=400, frequency='3MS')
    mimwrite('gif_%s.gif' % l, dataset, format='gif', fps=6)

In [ ]:
layers=['AIRS_Surface_Air_Temperature_Monthly_Day',
        'MERRA2_Surface_Skin_Temperature_Monthly',
        'MODIS_Terra_L3_SST_Thermal_9km_Day_Monthly',
        'MODIS_Aqua_L3_SST_Thermal_9km_Day_Monthly',
        'AIRS_Surface_Relative_Humidity_Monthly_Day',
        'MERRA2_Incident_Shortwave_Over_Land_Monthly',
        'MERRA2_Evaporation_from_Turbulence_Monthly',
        'MISR_Directional_Hemispherical_Reflectance_Average_Natural_Color_Monthly',
        'MERRA2_Carbon_Monoxide_Emission_Monthly'
        ]

In [ ]:
start_date = '2000-01-01'
end_date = '2019-01-01'

for l in layers:
    print('\n', l)
    dataset = collect_dataset(start_date, end_date, l, width=400, frequency='12MS')
    mimwrite('gif_%s.gif' % l, dataset, format='gif', fps=6)

In [ ]:
start_date = '2018-01-01'
end_date = '2019-01-01'

In [ ]:
layer = 'VIIRS_SNPP_CorrectedReflectance_TrueColor'
extent = (45.8,5.9,47.8,10.5)
dataset = collect_dataset(start_date, end_date, layer, extent=extent, width=400, frequency='1D')

In [ ]:
timeline = pd.date_range(start_date, end_date, freq='1W')
timeline.shape

In [ ]:
dataset.shape

In [ ]:
plt.imshow(np.min(dataset, axis=0)/255.)

In [ ]:
df = pd.DataFrame(dataset.reshape(dataset.shape[0], -1))
df.shape

In [ ]:
window_size = 16

In [ ]:
%time new = df.rolling(window_size).quantile(0.01)

In [ ]:
%time ok = new.to_numpy().reshape(dataset.shape).astype('uint8')

In [ ]:
plt.imshow(ok[200])

In [ ]:
#%time mimwrite('gif_orig.gif', dataset, format='gif', fps=30)

In [ ]:
%time mimwrite('gif_ok.gif', ok, format='gif', fps=30)